# QUANTITATIVE VALUE STRATEGY

**Value investing** refers to investing in the stocks which are cheapest relative to common measures of business value (like earnings or assets).

Here, an investing strategy is built that selects the 50 stocks with the best value metrics. Further calculating the recommended trades for an equal-weight portfolio of these 50 stocks high value stocks.

A basket of valuation metrics to build a robust quantitive value strategies is considered and stocks are filtered with respect to the lowest percentiles on the following metrics:

* **Price-to-earnings ratio**: It is the ratio for valuing a company that measures its current share price relative to its earnings per share (EPS). It indicates the dollar amount an investor can expect to invest in a company in order to receive $1 of that company's earnings. They determine the relative value of a company's shares in an apples-to-apples comparison and can also be used to compare a company against its own historical record or to compare aggregate markets against one another or over time.

* **Price-to-book ratio**: It is a financial ratio used to compare a company’s current market price to its book value. Companies use this ratio to compare a firm's market capitalization to its book value. It is calculated by dividing the company's stock price per share by its book value per share (BVPS). An asset's book value is equal to its carrying value on the balance sheet, and companies calculate it netting the asset against its accumulated depreciation.

* **Price-to-sales ratio**: It is a valuation ratio that compares a company’s stock price to its revenues. It is an indicator of the value that financial markets have placed on each dollar of a company’s sales or revenues.

* **Enterprise Value (EV)** : Market Cap plus Net debt plus Minorities.
It is a valuation metric for stocks and is calculated by dividing the company’s market capitalization by the revenue in the most recent year; or, equivalently, divide the per-share stock price by the per-share revenue. It is a measure of a company's total value, often used as a more comprehensive alternative to equity market capitalization. It includes in its calculation the market capitalization of a company but also short-term and long-term debt as well as any cash on the company's balance sheet.

* **EBITDA** : Reported Earnings Before Interest, Tax, Depreciation and Amortization.

* **EV/GP** :Enterprise Value divided by Gross Profit

These metrics are computed after pulling raw data.

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import stats
from scipy.stats import percentileofscore as pctile
from statistics import mean

# Requests library is extremely useful for HTTPS requests in Python
import requests

In [2]:
# To extract data from the Cloud API

# !pip install iexfinance
from iexfinance.stocks import Stock

# Data Extraction

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

# List of Stock data required
stocks=pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Batch API calls

Data is extracted from IEX API Sandbox by making Batch calls to decrease time complexity.

In [4]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

print(symbol_groups)

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])


[0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
      ...  
95     CINF
96       CL
97      CLX
98      CMA
99    CMCSA
Name: Ticker, Length: 100, dtype: object, 100     CME
101     CMG
102     CMI
103     CMS
104     CNC
       ... 
195    FTNT
196     FTV
197      GD
198      GE
199    GILD
Name: Ticker, Length: 100, dtype: object, 200     GIS
201      GL
202     GLW
203      GM
204    GOOG
       ... 
295     MAA
296     MAR
297     MAS
298     MCD
299    MCHP
Name: Ticker, Length: 100, dtype: object, 300     MCK
301     MCO
302    MDLZ
303     MDT
304     MET
       ... 
395     RHI
396     RJF
397      RL
398     RMD
399     ROK
Name: Ticker, Length: 100, dtype: object, 400     ROL
401     ROP
402    ROST
403     RSG
404     RTX
       ... 
495    XLNX
496     XOM
497    XRAY
498     XRX
499     XYL
Name: Ticker, Length: 100, dtype: object, 500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, dtype: object]


In [5]:
robust_value_cols = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',            # Final Results
    'Price-to-Earnings Ratio',            # Stores the Price-to-Earnings Ratio from the available IEX Cloud data
    'PE Percentile',                      # Calculates and stores the Price-to-Earnings Percentile
    'Price-to-Book Ratio',                # Stores the Price-to-Book Ratio from the available IEX Cloud data
    'PB Percentile',                      # Calculates and stores the Price-to-Book Percentile
    'Price-to-Sales Ratio',               # Stores the Price-to-Sales Ratio from the available IEX Cloud data
    'PS Percentile',                      # Calculates and stores the Price-to-Sales Percentile
    'EV/EBITDA',                          # Stores the EV/EBITDA from the available IEX Cloud data
    'EV/EBITDA Percentile',               # Calculates and stores the EV/EBITDA Percentile
    'EV/GP',                              # Stores the EV/GP from the available IEX Cloud data
    'EV/GP Percentile',                   # Calculates and stores the EV/GP Percentile
    'Robust Value Score'                            # Stores the Robust Value Scores
]

rv_df = pd.DataFrame(columns = robust_value_cols)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_df = rv_df.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = robust_value_cols),
            ignore_index = True
        )

C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append 

C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append 

C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append 

C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_df = rv_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_18116\2207086206.py:38: FutureWarning: The frame.append 

KeyError: 'DISCA'

# Data Preprocessing

## Missing Data in Dataframe

In [6]:
# Columns with NaN values
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Robust Value Score
28,ALXN,189.3,N/A,61.48,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,311.09,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,304.18,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,96.63,N/A,48.19,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11,N/A,10.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [7]:
for i in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_df[i].fillna(rv_df[i].mean(), inplace = True)

# High Quality Value Percentiles

Calculating value score percentiles (following metrics) for every stock in the universe.

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

In [8]:
metrics = { 'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_df.index:
    for metric in metrics.keys():
        rv_df.loc[row, metrics[metric]] = stats.percentileofscore(rv_df[metric], rv_df.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_df[metric])

#Print the entire DataFrame    
rv_df

0      0.703704
1      0.044444
2      0.525926
3      0.651852
4      0.459259
         ...   
130    0.585185
131    0.214815
132    0.081481
133    0.711111
134    0.948148
Name: PE Percentile, Length: 135, dtype: object
0      0.748148
1       0.02963
2      0.525926
3       0.97037
4      0.903704
         ...   
130     0.77037
131    0.325926
132    0.111111
133     0.57037
134    0.318519
Name: PB Percentile, Length: 135, dtype: object
0      0.822222
1      0.022222
2      0.162963
3      0.859259
4      0.696296
         ...   
130    0.288889
131    0.259259
132    0.118519
133    0.844444
134    0.481481
Name: PS Percentile, Length: 135, dtype: object
0      0.814815
1           1.0
2      0.377778
3       0.77037
4      0.318519
         ...   
130    0.622222
131    0.192593
132    0.074074
133    0.807407
134    0.718519
Name: EV/EBITDA Percentile, Length: 135, dtype: object
0      0.822222
1      0.037037
2      0.177778
3      0.948148
4      0.548148
         ...   
1

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Robust Value Score
0,A,139.13,N/A,34.08,0.703704,8.36,0.748148,6.5000,0.822222,24.082726,0.814815,12.065115,0.822222,N/A
1,AAL,14.45,N/A,-5.00,0.044444,-1.11,0.02963,0.2267,0.022222,107.075856,1.0,0.824966,0.037037,N/A
2,AAP,216.31,N/A,23.37,0.525926,4.35,0.525926,1.1800,0.162963,12.482096,0.377778,2.648623,0.177778,N/A
3,AAPL,173.61,N/A,29.42,0.651852,48.15,0.97037,7.3700,0.859259,22.381391,0.77037,17.331244,0.948148,N/A
4,ABBV,145.51,N/A,20.69,0.459259,17.63,0.903704,4.4900,0.696296,11.118353,0.318519,7.740540,0.548148,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,DG,257.64,N/A,26.95,0.585185,9.79,0.77037,1.7100,0.288889,17.345631,0.622222,5.821450,0.377778,N/A
131,DGX,141.67,N/A,11.46,0.214815,2.54,0.325926,1.5400,0.259259,8.526752,0.192593,4.866416,0.311111,N/A
132,DHI,74.93,N/A,4.82,0.081481,1.49,0.111111,0.8476,0.118519,3.978235,0.074074,2.814507,0.192593,N/A
133,DHR,304.75,N/A,35.03,0.711111,4.94,0.57037,7.2000,0.844444,23.218724,0.807407,12.179696,0.82963,N/A


# The Robust Value Score

Robust Value Score is the Arithmetic mean of the 5 percentile scores used above.

In [9]:
for row in rv_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'Robust Value Score'] = mean(value_percentiles)
    
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Robust Value Score
0,A,139.13,N/A,34.08,0.703704,8.36,0.748148,6.5000,0.822222,24.082726,0.814815,12.065115,0.822222,0.782222
1,AAL,14.45,N/A,-5.00,0.044444,-1.11,0.02963,0.2267,0.022222,107.075856,1.0,0.824966,0.037037,0.226667
2,AAP,216.31,N/A,23.37,0.525926,4.35,0.525926,1.1800,0.162963,12.482096,0.377778,2.648623,0.177778,0.354074
3,AAPL,173.61,N/A,29.42,0.651852,48.15,0.97037,7.3700,0.859259,22.381391,0.77037,17.331244,0.948148,0.84
4,ABBV,145.51,N/A,20.69,0.459259,17.63,0.903704,4.4900,0.696296,11.118353,0.318519,7.740540,0.548148,0.585185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,DG,257.64,N/A,26.95,0.585185,9.79,0.77037,1.7100,0.288889,17.345631,0.622222,5.821450,0.377778,0.528889
131,DGX,141.67,N/A,11.46,0.214815,2.54,0.325926,1.5400,0.259259,8.526752,0.192593,4.866416,0.311111,0.260741
132,DHI,74.93,N/A,4.82,0.081481,1.49,0.111111,0.8476,0.118519,3.978235,0.074074,2.814507,0.192593,0.115556
133,DHR,304.75,N/A,35.03,0.711111,4.94,0.57037,7.2000,0.844444,23.218724,0.807407,12.179696,0.82963,0.752593


# Selection of the best 50 Value Stocks

The best 50 stocks are calculated by sorting the dataframe with respect to robust value scores of each stock.

In [10]:
rv_df.sort_values(by = 'Robust Value Score', inplace = True)
rv_df = rv_df[:50]
rv_df.reset_index(drop = True, inplace = True)

Portfolio size the investor is willing to invest.

In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        value = float(portfolio_size)
    except ValueError:                                       
      # If the user does not enter the numericals
        print("This is not a number! \n Please try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:100000
100000


## Number of Shares to Buy

Strategy: Investing the the top 50 high value stocks

In [12]:
amt = float(portfolio_size)/len(rv_df.index)
for i in range(len(rv_df['Ticker'])):
    rv_df.loc[i, 'Number of Shares to Buy']=math.floor(amt/rv_df['Price'][i])

In [13]:
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Robust Value Score
0,AIG,57.2,34,3.72,0.066667,0.9670,0.059259,0.7569,0.103704,2.169792,0.02963,0.728004,0.02963,0.057778
1,CAH,71.99,27,-20.82,0.022222,-27.8400,0.007407,0.1060,0.007407,7.194667,0.17037,2.850151,0.2,0.081481
2,COF,115.8,17,5.10,0.088889,0.8263,0.044444,1.3100,0.207407,3.561481,0.044444,1.336440,0.081481,0.093333
3,C,53.75,37,6.77,0.111111,0.5820,0.037037,1.2400,0.17037,4.047125,0.081481,1.207025,0.074074,0.094815
4,DHI,74.93,26,4.82,0.081481,1.4900,0.111111,0.8476,0.118519,3.978235,0.074074,2.814507,0.192593,0.115556
5,AIZ,172.64,11,8.27,0.125926,2.1400,0.266667,0.8956,0.125926,3.743201,0.059259,0.886280,0.059259,0.127407
6,BEN,28.43,70,8.57,0.133333,1.3000,0.088889,1.6900,0.281481,5.914686,0.125926,1.498680,0.096296,0.145185
7,AFL,64.78,30,9.63,0.162963,1.5500,0.118519,2.0400,0.355556,6.429363,0.133333,1.962324,0.118519,0.177778
8,CVS,107.45,18,17.40,0.37037,1.8300,0.192593,0.4631,0.044444,9.089665,0.214815,1.464074,0.088889,0.182222
9,CCL,10.3,194,-1.25,0.059259,1.3800,0.103704,1.8700,0.318519,-10.410671,0.022222,6.783211,0.422222,0.185185


# Formatting the output in excel

Converting the recommended shares based on momentum of stocks to an excel file.

In [ ]:
rv_df.to_excel('Recommended_Shares_based_on_value.xlsx')